<a href="https://colab.research.google.com/github/dmswl0707/CoronaCoprosperityCard_DataScience/blob/main/Card_Payment_Ratio_by_Age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import json

from google.colab import drive
drive.mount('/content/drive') #구글 드라이브에 접근

filename1='/content/drive/My Drive/Colab Notebooks/data/card_2020.csv' #파일 경로설정
data1=pd.read_csv(filename1) #csv 파일 불러오기
print(data1.head())
print(data1.info()) #파일정보

In [ ]:
filename2='/content/drive/My Drive/Colab Notebooks/data/index.csv' #파일 경로설정
data2=pd.read_csv(filename2) #csv 파일 불러오기
print(data2.head())
print(data2.info()) #파일정보

In [ ]:
np.sum(pd.isnull(data1)) #결측치 확인
np.sum(pd.isnull(data2))

In [ ]:
###필요한 index 추출
 
data1_sample=data1[['mrhst_induty_cl_nm','salamt']]
print(data1_sample.head(100))

data2_sample=data2[['age','catl','catm']]
print(data2_sample.head(100))

In [ ]:
#1. data1에서 결제건수와 판매업종 추출
#2. data2에서 나이와 이용업종 추출 (판매업종=이용업종)
#3. 반복된 값 정리 -ing
#4. ex) 20 : (업종,1순위,),(업종,2순위),(업종,3순위),(업종,4순위),(업종,5순위) 
#   ex) 판매액, 판매건수 : (20,1위),(40,2위),(30,3위),(50,4위),(60,5위)
#5....소비성향(k-means)


#가게분류_list=data1['receipt_dttm']
#가게분류_val==가게분류_list.values.tolist()


##연령리스트, 각 행 비율
age_list=data2['age']
age_val=age_list.values.tolist()
print(age_val)
data2['age'].value_counts(normalize=True)